<a href="https://colab.research.google.com/github/fahmidjobbi/TP-COLLAB/blob/main/YES_NO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from numpy import expand_dims

from numpy import zeros

from numpy import ones

from numpy import vstack

from numpy.random import randn

from numpy.random import randint

from keras.datasets.mnist import load_data

from tensorflow.keras.optimizers import Adam

from keras.models import Sequential

from keras.layers import Dense

from keras.layers import Reshape

from keras.layers import Flatten

from keras.layers import Conv2D

from keras.layers import Conv2DTranspose

from keras.layers import LeakyReLU

from keras.layers import Dropout

from matplotlib import pyplot


In [3]:
import tensorflow as tf
print("Num GPUs Availble:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Availble: 1


In [4]:
import os
os.chdir('/content/drive/MyDrive/yes')
os.getcwd()

'/content/drive/MyDrive/yes'

In [5]:
import cv2
import glob
Data=[]
for i in glob.glob("/content/drive/MyDrive/yes/*"):
  image=cv2.imread(i, cv2.IMREAD_COLOR)
  bigger=cv2.resize(image, (80,80))
  Data.append(bigger)

In [10]:
import numpy as np
Data=np.array(Data)
type(Data)
Data.shape

(155, 80, 80, 3)

In [11]:
# define the standalone discriminator model

def define_discriminator(in_shape=(80,80,3)):

  model = Sequential()

  model.add(Conv2D(256, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))

  model.add(LeakyReLU(alpha=0.2))

  model.add(Dropout(0.4))

  model.add(Conv2D(256, (3,3), strides=(2, 2), padding='same'))

  model.add(LeakyReLU(alpha=0.2))

  model.add(Dropout(0.4))

  model.add(Flatten())

  model.add(Dense(1, activation='sigmoid'))

  # compile model

  opt = Adam(lr=0.0002, beta_1=0.5)

  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

In [12]:
discriminator=define_discriminator()
discriminator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 40, 40, 256)       7168      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 40, 40, 256)       0         
                                                                 
 dropout (Dropout)           (None, 40, 40, 256)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 256)       590080    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 20, 20, 256)       0         
                                                                 
 dropout_1 (Dropout)         (None, 20, 20, 256)       0         
                                                                 
 flatten (Flatten)           (None, 102400)            0

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [13]:
def define_generator(latent_dim):
   n_nodes= 256*20*20
   
   model = Sequential()
   model.add(Dense(n_nodes, input_dim=latent_dim))
   model.add(LeakyReLU(alpha=0.2))
   model.add(Reshape((20, 20, 256)))
   #upsample tp 14*14
   model.add(Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same'))
   model.add(LeakyReLU(alpha=0.2))
   #upsample to 28*28
   model.add(Conv2DTranspose(256, (4,4), strides=(2, 2), padding='same'))
   model.add(LeakyReLU(alpha=0.2))
   model.add(Conv2D(3, (20,20), activation='sigmoid', padding='same' ))
   return model


In [15]:
generator=define_generator(100)
generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 102400)            10342400  
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 102400)            0         
                                                                 
 reshape_1 (Reshape)         (None, 20, 20, 256)       0         
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 40, 40, 256)      1048832   
 ranspose)                                                       
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 40, 40, 256)       0         
                                                                 
 conv2d_transpose_3 (Conv2DT  (None, 80, 80, 256)      1048832   
 ranspose)                                            

In [16]:
from tensorflow._api.v2.config import optimizer
def define_gan(g_model, d_model):
  #make weights in the discriminator not trainable
  d_model.trainable=False
  #connect them
  model=Sequential()
  #add generator
  model.add(g_model)
  #add discriminator
  model.add(d_model)
  #compile mode
  opt= Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

In [17]:
#load and prepare mnist training images
def load_real_samples(Data1):
  X = Data1.astype('float32')
  X = X/255.0
  return X

In [20]:
X1=load_real_samples(Data)
X1.shape


(155, 80, 80, 3)